# the code

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from io import StringIO

# Set up Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Create a new browser session with headless Chrome
driver = webdriver.Chrome(options=chrome_options)

# Rest of your code remains the same
driver.get("http://www.krishimaratavahini.kar.nic.in/reports/DateWiseReport.aspx")

# Define lists of values for each dropdown
months = ["JANUARY"]
years = [str(year) for year in range(2022, 2023)]  # Years from 2002 to 2023
commodities = ["GROUNDNUT"]  # Add all the commodities you want to iterate through
markets = ["BALLARI"]  # Add all the markets you want to iterate through

# Initialize an empty list to store DataFrames
data_frames = []

# Iterate through the combinations
for year in years:
    for month in months:
        for commodity in commodities:
            for market in markets:
                # Select values in the dropdown menus
                month_dropdown = Select(driver.find_element(By.ID, "_ctl0_content5_ddlmonth"))
                month_dropdown.select_by_visible_text(month)

                year_dropdown = Select(driver.find_element(By.ID, "_ctl0_content5_ddlyear"))
                year_dropdown.select_by_visible_text(year)

                commodity_dropdown = Select(driver.find_element(By.ID, "_ctl0_content5_ddlcommodity"))
                commodity_dropdown.select_by_visible_text(commodity)

                market_dropdown = Select(driver.find_element(By.ID, "_ctl0_content5_ddlmarket"))
                market_dropdown.select_by_visible_text(market)

                # Click the "View Report" button
                view_button = driver.find_element(By.ID, "_ctl0_content5_viewreport")
                view_button.click()

                # Find the table element by its ID
                table = driver.find_element(By.ID, "_ctl0_content5_gv")

                # Get the HTML content of the table
                table_html = table.get_attribute("outerHTML")

                # Convert the HTML table to a Pandas DataFrame
                html_buffer = StringIO(table_html)
                df = pd.read_html(html_buffer)[0]  # Assuming it's the first (and only) table on the page

                # Append the DataFrame to the list
                data_frames.append(df)

                # Go back to the main page to select the next combination
                driver.back()

# Close the browser session
driver.quit()

# Now you have a list of DataFrames, one for each combination
# You can concatenate them into a single DataFrame if needed
final_dataframe = pd.concat(data_frames, ignore_index=True)

## basic pre-processing for the scraped data

In [ ]:
# Filter out rows containing 'Total' in any column
df = final_dataframe[~final_dataframe.apply(lambda row: row.astype(str).str.contains('Total').any(), axis=1)].copy()

# Fill missing values in 'Market' column with the last non-null value
df.loc[:, 'Market'] = df['Market'].ffill()

# Convert the 'Date' column to a datetime object with day-first format
df.loc[:, 'Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)